In [1]:
import numpy as np

# For plotting
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

import deepxde as dde
dde.backend.set_default_backend("tensorflow")
import deepxde.backend as tf

2024-03-19 13:07:23.466908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


Setting the default backend to "tensorflow". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


In [ ]:
t_max = 5
R = 5
H = 3
h = 2
A = 0.01
mu = 1
mu_z = 10

r_min = 0.01
r_max = R
z_min = 0
z_max = H

t_points = 64
r_points = 64
z_points = 64

# Creation of the 2D domain (for plotting and input)
r = np.linspace(r_min, r_max, r_points)
z = np.linspace(z_min, z_max, z_points)
r, z = np.meshgrid(r, z)

space_domain = dde.geometry.Rectangle([r_min, z_min], [r_max, z_max])
time_domain = dde.geometry.TimeDomain(0, t_max)
domain = dde.geometry.GeometryXTime(space_domain, time_domain)

In [ ]:
def pde(x, y):
    """
    INPUTS:
        x: x[:,0] is r-coordinate
           x[:,1] is z-coordinate
           x[:,2] is t-coordinate
        y: Network output, in this case:
            y[:,0] is v_r(r, z, t) the speed for r component
            y[:,1] is v_z(r, z, t) the speed for z component
            y[:,1] is p(r, z, t) the pressure
    OUTPUT:
        The pde in standard form i.e. something that must be zero
    """
    r, z, t = x[:,0], x[:, 1], x[:, 2]
    vr, vz, p = y[:,0], y[:, 1], y[:, 2]
    dvr_t = dde.grad.jacobian(y, x, i=0, j=2)
    dvz_t = dde.grad.jacobian(y, x, i=1, j=2)
    dvr_r = dde.grad.jacobian(y, x, i=0, j=0)
    dvz_r = dde.grad.jacobian(y, x, i=1, j=0)
    dvr_z = dde.grad.jacobian(y, x, i=0, j=1)
    dvz_z = dde.grad.jacobian(y, x, i=1, j=1)
    dp_r =  dde.grad.jacobian(y, x, i=2, j=0)
    dp_z =  dde.grad.jacobian(y, x, i=2, j=1)

    d2vr_z2 = dde.grad.hessian(y, x, component=0, i=1, j=1)
    d2vz_z2 = dde.grad.hessian(y, x, component=1, i=1, j=1)
    d2vr_r2 = dde.grad.hessian(y, x, component=0, i=0, j=0)
    d2vz_r2 = dde.grad.hessian(y, x, component=1, i=0, j=0)
    
    f_vr = dvr_t + vr * dvr_r + vz * dvr_z + 1 / R * dp_r - mu * (1 / r * dvr_r + d2vr_r2 + d2vr_z2 - vr / r ** 2)
    f_vz = dvz_t + vr * dvz_r + vz * dvz_z + 1 / R * dp_z - mu * (1 / r * dvz_r + d2vz_r2 + d2vz_z2)
    f_continuity = dvr_r + vr / r + dvz_z

    return [f_vr, f_vz, f_continuity]

In [ ]:
# Boundary and Initial conditions
def boundary(x, on_boundary):
    if x[0] == r_min:
        return False
    return on_boundary

def boundary_r(x):
    return 0

def boundary_z(x):
    r, z, t = x[:,0], x[:, 1], x[:, 2]
    return A * np.cos(np.pi * r / (2 * R)) * np.sin(2 * np.pi * mu_z)


# Initial conditions
def init_cond_z(x):
    return 0

def init_cond_r(x):
    return 0

bc_r_0 = dde.icbc.DirichletBC(domain, boundary_r, boundary, component=0)
bc_z_0 = dde.icbc.DirichletBC(domain, boundary_z, boundary, component=1)

bc_r_1 = dde.icbc.NeumannBC(domain, boundary_r, boundary, component=0)
bc_z_1 = dde.icbc.NeumannBC(domain, boundary_z, boundary, component=1)

ic_r = dde.icbc.IC(domain, init_cond_r, boundary, component=0)
ic_z = dde.icbc.IC(domain, init_cond_z, boundary, component=1)



In [ ]:
data = dde.data.TimePDE(domain, pde, [bc_r_0, bc_z_0, ic_r, ic_z],
                         num_domain=r_points * z_points,
                         num_boundary=z_points,
                         num_initial=r_points,
                         train_distribution="pseudo",
)

# Network architecture
net = dde.nn.FNN([3] + [32] * 4 + [3], "tanh", "Glorot normal")
model = dde.Model(data, net)

In [ ]:
# To employ a GPU accelerated system is highly encouraged.

model.compile("adam", lr=1e-3, loss="MSE")
model.train(iterations=1000, display_every=1)

In [ ]:
model.losshistory